# ✔ This is an Advanced Python Project called Slot Machine Game (Casino Slot Simulator)

#                               Python Slot Machine Simulator with GUI & Animation

In [69]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import random
import os
import sys

In [70]:
# CONSTANTS

ROWS = 3
COLS = 3
MAX_LINES = 3
MIN_BET = 1
MAX_BET = 100

SYMBOL_COUNT = {"A": 2, "B": 4, "C": 6, "D": 8}
SYMBOL_VALUE = {"A": 5, "B": 4, "C": 3, "D": 2}

BG = "#1a1a1a"
CARD = "#2a2a2a"
TEXT = "white"
ACCENT = "#4CAF50"

class SlotMachineGUI:

    def __init__(self, root):
        self.root = root
        self.root.title("🎰 Casino Slot Machine – Python GUI")
        self.root.geometry("500x600")
        self.root.configure(bg=BG)

        self.balance = 0
        self.bet = 1
        self.lines = 1

        self.build_header()
        self.build_controls()
        self.build_slots()
        self.build_footer()

   
    # HEADER
    
    def build_header(self):
        tk.Label(self.root, text="🎰 Slot Machine Game",
                 bg=BG, fg=TEXT, font=("Arial", 22, "bold")).pack(pady=10)

    # -----------------------------
    # CONTROLS
    # -----------------------------
    def build_controls(self):
        frame = tk.Frame(self.root, bg=CARD)
        frame.pack(pady=10)

        # Balance
        self.balance_label = tk.Label(frame, text="Balance: $0",
                                      font=("Arial", 14), bg=CARD, fg=TEXT)
        self.balance_label.grid(row=0, column=0, columnspan=3, pady=10)

        tk.Button(frame, text="Deposit", command=self.deposit_money,
                  font=("Arial", 12), bg=ACCENT, fg="white").grid(row=0, column=3, padx=10)

        # Bet Controls
        tk.Label(frame, text="Bet/Line:", bg=CARD, fg=TEXT, font=("Arial", 12)).grid(row=1, column=0)
        self.bet_label = tk.Label(frame, text="$1", bg=CARD, fg=TEXT, font=("Arial", 12))
        self.bet_label.grid(row=1, column=1)
        tk.Button(frame, text="+", command=self.inc_bet, width=3).grid(row=1, column=2)
        tk.Button(frame, text="-", command=self.dec_bet, width=3).grid(row=1, column=3)

        # Lines
        tk.Label(frame, text="Lines:", bg=CARD, fg=TEXT, font=("Arial", 12)).grid(row=2, column=0)
        self.line_label = tk.Label(frame, text="1", bg=CARD, fg=TEXT, font=("Arial", 12))
        self.line_label.grid(row=2, column=1)
        tk.Button(frame, text="+", command=self.inc_lines, width=3).grid(row=2, column=2)
        tk.Button(frame, text="-", command=self.dec_lines, width=3).grid(row=2, column=3)

    
    # SLOT MACHINE GRID
    
    def build_slots(self):
        self.slot_frame = tk.Frame(self.root, bg=CARD)
        self.slot_frame.pack(pady=20)

        self.labels = [[tk.Label(self.slot_frame, text="-", font=("Arial", 28),
                                 width=4, bg="#333", fg="white")
                        for _ in range(COLS)] for _ in range(ROWS)]

        for r in range(ROWS):
            for c in range(COLS):
                self.labels[r][c].grid(row=r, column=c, padx=10, pady=5)

    # -----------------------------
    # FOOTER
    # -----------------------------
    def build_footer(self):
        tk.Button(self.root, text="SPIN 🎲", font=("Arial", 18, "bold"),
                  bg=ACCENT, fg="white", width=12,
                  command=self.spin).pack(pady=20)

    
    # GAME LOGIC

    def deposit_money(self):
        amount = simpledialog.askinteger("Deposit", "Enter amount:")
        if amount and amount > 0:
            self.balance += amount
            self.update_balance()

    def inc_bet(self):
        if self.bet < MAX_BET:
            self.bet += 1
            self.bet_label.config(text=f"${self.bet}")

    def dec_bet(self):
        if self.bet > MIN_BET:
            self.bet -= 1
            self.bet_label.config(text=f"${self.bet}")

    def inc_lines(self):
        if self.lines < MAX_LINES:
            self.lines += 1
            self.line_label.config(text=self.lines)

    def dec_lines(self):
        if self.lines > 1:
            self.lines -= 1
            self.line_label.config(text=self.lines)

    def update_balance(self):
        self.balance_label.config(text=f"Balance: ${self.balance}")

    
    # SPIN LOGIC + ANIMATION
    
    def spin(self):
        total_bet = self.bet * self.lines

        if total_bet > self.balance:
            messagebox.showerror("Error", "Not enough balance!")
            return

        self.balance -= total_bet
        self.update_balance()

        # Animation effect
        for _ in range(10):
            for r in range(ROWS):
                for c in range(COLS):
                    self.labels[r][c].config(text=random.choice(list(SYMBOL_COUNT.keys())))
            self.root.update()
            self.root.after(50)

        # Final symbols
        columns = self.get_spin_result()

        for r in range(ROWS):
            for c in range(COLS):
                self.labels[r][c].config(text=columns[c][r])

        winnings, win_lines = self.check_winnings(columns)
        self.balance += winnings
        self.update_balance()

        if winnings > 0:
            messagebox.showinfo("Win!", f"You won ${winnings} on lines {win_lines}")
        else:
            messagebox.showinfo("Result", "No win. Try again!")

    
    # MACHINE LOGIC
    
    def get_spin_result(self):
        all_symbols = []
        for symbol, count in SYMBOL_COUNT.items():
            all_symbols.extend([symbol] * count)

        columns = []
        for _ in range(COLS):
            column = []
            current = all_symbols[:]
            for _ in range(ROWS):
                value = random.choice(current)
                current.remove(value)
                column.append(value)
            columns.append(column)
        return columns

    def check_winnings(self, columns):
        winnings = 0
        winning_lines = []

        for line in range(self.lines):
            first_symbol = columns[0][line]
            for column in columns:
                if column[line] != first_symbol:
                    break
            else:
                winnings += SYMBOL_VALUE[first_symbol] * self.bet
                winning_lines.append(line + 1)

        return winnings, winning_lines



# RUN APP

root = tk.Tk()
app = SlotMachineGUI(root)
root.mainloop()
